In [1]:
import pandas as pd
import numpy as np
import sys, os, re
from numba import njit, typed
from numba.experimental import jitclass

if re.match(r'^(\w\:\\)|(/)', os.getcwd()):
    sys.path.insert(0, "c:/Users/dmitr/devs/quantl/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/src/")
    # sys.path.insert(0, "c:/Users/dmitr/devs/Qube/")
    print('installed')

installed


# First experiments

In [2]:
# Time data representation
from qube.quantitative.ta.indicators import ema, _calc_ema
from qube.quantitative.tools import rolling_sum
from qube.quantitative.tools import column_vector

from quantl.core import Indicator
from quantl.impl.indicators import Sma, Ema
from quantl.utils import nans

ModuleNotFoundError: No module named 'qube.quantitative'

In [131]:
class SmaNu(Indicator):
    """
    Simple moving average
    """
    def __init__(self, period):
        super().__init__()
        self.period = period
        self.__s = nans(period)
        self.__i = typed.List([0])
        self._name = 'sma(%d)' % period
        self._calculate = self.wrap(period) 

    def wrap(cls, period):
        @njit
        def _do_calc(x, _is_being_appended, __s, __i):
            _x = x / period
            __s[__i[0]] = _x

            if _is_being_appended:
                __i[0] += 1
                if __i[0] >= period:
                    __i[0] = 0

            return np.sum(__s)
        return _do_calc

    def calculate(self, x):
        # print(self.__s)
        return self._calculate(
            x, self._is_being_appended, self.__s, self.__i
        )

In [215]:
class EmaFast(Indicator):
    def __init__(self, period, init_mean=True):
        super().__init__()
        self.period = period
        self.init_mean = init_mean
        self._name = 'ema(%d,%s)' % (period, init_mean)
        self.xs = [] 
        self.xc = np.arange(1500).reshape(-1, 1)

    def calculate(self, x):
        # self.xs.append(x)
        # xc = np.array(self.xs).reshape(-1, 1)
        # return 0
        return _calc_ema(self.xc, self.period, self.init_mean)[-1][0]

In [3]:
%%timeit

e0 = Ema(15)
for i in range(1500):
    e0.update(i, i)

2.47 ms ± 438 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [216]:
%%timeit

e1 = EmaFast(15)
for i in range(1500):
    e1.update(i, i)

16.7 ms ± 559 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [154]:
%%timeit

sma1 = SmaNu(15)
for i in range(1000):
    sma1.update(i, float(i))

116 ms ± 4.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [148]:
%%timeit
sma0 = Sma(15)
for i in range(1000):
    sma0.update(i, i)

4.27 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Temporal data representation

In [121]:
%load_ext autoreload
%autoreload 2

from qube.utils import infer_series_frequency
from qube.core.series import BarSeries, recognize_time, TimeSeries, FloatSeries, Float, Trade

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
infer_series_frequency([
    recognize_time('2020-01-01 10:00:01'), 
    recognize_time('2020-01-01 10:00:02'),
    recognize_time('2020-01-01 10:00:05'),
    recognize_time('2020-01-01 10:00:14'),
])

In [6]:
# b = BarSeries(time=[10, 20, 30], data=[[90, 100, 80, 90], [91, 101, 81, 91], [92, 102, 82, 92]])

In [ ]:
# b._item_at(-1)

In [122]:
# print(b)

In [123]:
fs = FloatSeries('close', '10Min')
fs.update(Float(recognize_time('2024-01-01 00:00'), 1))
fs.update(Float(recognize_time('2024-01-01 00:01'), 2))
fs.update(Float(recognize_time('2024-01-01 00:12'), 3))
fs.update(Float(recognize_time('2024-01-01 00:21'), 4))
fs.update(Float(recognize_time('2024-01-01 00:22'), 5))
fs.update(Float(recognize_time('2024-01-01 00:31'), 6))
fs.update(Float(recognize_time('2024-01-01 00:33'), 7))
fs.update(Trade(recognize_time('2024-01-01 00:45'), 12, 100))
fs.update(Trade(recognize_time('2024-01-01 00:55'), 13, 100))

True

In [ ]:
pd.date_range()

In [124]:
print(str(fs))

TimeSeries[close / 10 minutes] | 6 records
  [2024-01-01T00:00:00]	2 
  [2024-01-01T00:10:00]	3 
  [2024-01-01T00:20:00]	5 
   .......... 
  [2024-01-01T00:20:00]	5 
  [2024-01-01T00:10:00]	3 
  [2024-01-01T00:00:00]	2 



In [126]:
fs[0:3]

[[2024-01-01T00:50:00]	13 ,
 [2024-01-01T00:40:00]	12 ,
 [2024-01-01T00:30:00]	7 ]